In [1]:
import pandas as pd
import numpy as np

from route import Route
from weather_forecast import WeatherForecast

In [2]:
current_position = {'longitude': 130.868566,
                    'latitude': -12.432466}
delta_spacing = 100000.0 # in meters
number_sites = 3

In [3]:
route = Route()
# route.get_final_data(current_position)
# route.get_final_data(current_position, delta_spacing=delta_spacing)
dataframe = route.get_final_data(current_position, number_sites=number_sites)

    longitude   latitude    altitude   cumDistance  gradient  maxSpeed
0  130.868566 -12.432466   11.500000  0.000000e+00  0.024056       100
1  133.803762 -23.861845  557.690488  1.513018e+06  0.004450       130
2  138.608989 -34.935232   53.250000  3.026035e+06 -0.017856        50


In [4]:
api = WeatherForecast()

Response status from siteinfo: OK.
Current sites:
  name site_id  longitude  latitude
0   cc  584867    130.869  -12.4325
1    1  584868    130.000  -12.0000
2   cc  584869    130.000  -12.0000
3    0  584873    130.869  -12.4325
4    1  584874    133.804  -23.8618
5    2  584875    138.609  -34.9352


In [5]:
# api.add_sites(dataframe)

In [6]:
position = {
    'latitude': -12.0,
    'longitude': 130.0
}
api.get_site_edit(584867, True, name="cc")
api.get_site_edit(584868, True, position=position)
api.get_site_edit(584869, True, name="cc", position=position)

Response status from siteedit: OK (200).
Site with ID 584867 has been edited: New name: cc. 
Response status from siteedit: OK (200).
Site with ID 584868 has been edited: New position: {'latitude': -12.0, 'longitude': 130.0}. 
Response status from siteedit: OK (200).
Site with ID 584869 has been edited: New name: cc. New position: {'latitude': -12.0, 'longitude': 130.0}. 


In [7]:
api.get_site_info()

Response status from siteinfo: OK.
{
  "status": "OK",
  "statuscode": 200,
  "payload": {
    "solarforecast": {
      "sites": {
        "584867": {
          "altitude": 18,
          "azimuth": 0,
          "cddctout": null,
          "hddctin": null,
          "hddctout": null,
          "horizon": [
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0
          ],
          "id": 584867,
          "inclination": 0,
          "last_update": "2023-08-23 13:32:37",
          "latitude": -12.4325,
          "longitude": 130.869,
          "name": "cc",
          "registered": "2023-08-23 11:56:00",
          "timezone": "Australia/Darwin",
          "utc_offset": "UTC+9.5"
        },
        "584868": {
          "altitude": 557,
          "azimuth": 0,
          "cddctout": null,
          "hddctin": null,
          "hddctout": null,
          "horizon": [
  

<Response [200]>

In [8]:
api.delete_all_sites()

Response status from sitedelete: OK (200).
Site with id 584867 has been removed.
Response status from sitedelete: OK (200).
Site with id 584868 has been removed.
Response status from sitedelete: OK (200).
Site with id 584869 has been removed.
Response status from sitedelete: OK (200).
Site with id 584873 has been removed.
Response status from sitedelete: OK (200).
Site with id 584874 has been removed.
Response status from sitedelete: OK (200).
Site with id 584875 has been removed.
All sites have been deleted.
Empty DataFrame
Columns: [name, site_id, longitude, latitude]
Index: []


In [9]:
api.get_site_info()

Response status from siteinfo: OK.
{
  "status": "OK",
  "statuscode": 200,
  "payload": {
    "solarforecast": {
      "sites": [],
      "user": {
        "id": 393,
        "name": "aCentauri"
      }
    }
  }
}


<Response [200]>